In [ ]:
import multiprocessing
import os
import fsspec
import pandas as pd
import xarray as xr
from tqdm import tqdm

#https://noaa-nwm-retro-v2-zarr-pds.s3.amazonaws.com/index.html
#https://noaa-nwm-retrospective-2-1-zarr-pds.s3.amazonaws.com/index.html
#        noaa-nwm-retrospective-2-1-zarr-pds/
#https://noaa-nwm-retrospective-2-1-pds.s3.amazonaws.com/index.html



# #%%
# usgs_stations = pd.read_csv(
# # change the path to the path of the csv file: NWM_USGS_Artifical_Path.csv or NWM_USGS_Natural_Flow.csv
#     '/home/ismart/usgs_test.csv')
# usgs_stations['USGS_ID'] = usgs_stations['USGS_ID'].apply(lambda x: '{:0>8}'.format(x))
# #%%
fs = fsspec.filesystem('s3', anon=True)
_file = fs.glob('noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr')

ds = xr.open_dataset(fs.get_mapper(_file[0]), engine='zarr', backend_kwargs={'consolidated': True})


In [ ]:
#%%
def download_nwm_stations(row):
    try:
        print('\nRunning station row : ', row, ' | NWM_ID : ', usgs_stations.loc[row, 'NWM_ID'], ' | USGS_ID : ',
              usgs_stations.loc[row, 'USGS_ID'],'...')
        Forecast_point = ds['streamflow'].sel(feature_id=int(usgs_stations.loc[row, 'NWM_ID']))
        # Forecast_point = ds['velocity'].sel(feature_id=int(usgs_stations.loc[row, 'NWM_ID']))
        Forecast_point = Forecast_point.to_pandas().to_frame()
        Forecast_point.columns = [usgs_stations.loc[row, 'NWM_ID']]
        # Forecast_point.to_csv(os.path.join(os.path.dirname(os.path.abspath(__file__)), 'data',
        #                                    str(usgs_stations.loc[row, 'NWM_ID']) + '.csv'))
        Forecast_point.to_csv(os.path.join('/home/ismart', str(usgs_stations.loc[row, 'NWM_ID']) + '.csv'))
        
    except:
        print('\nError with station row : ', row, ' | NWM_ID : ', usgs_stations.loc[row, 'NWM_ID'], ' | USGS_ID : ',
              usgs_stations.loc[row, 'USGS_ID'])
        pass
    return 0


def run(usgs_stations):
    print('Getting list of files...')
    files = [e for e in usgs_stations.index]
    pool = multiprocessing.Pool(processes=6)
    print('Downloading NWM Stations ...')
    _stats = tqdm(pool.map(download_nwm_stations, files), desc='Processing rows', total=len(files))

if __name__ == '__main__':
    run(usgs_stations)